In [1]:
import numpy as np
import gdspy
from autograd import numpy as npa
from scipy.ndimage import binary_fill_holes

################################################### design parameter #########################
resolution = 50  # 해상도
design_region_width = 3  # 디자인 영역 너비
design_region_height = 4  # 디자인 영역 높이
pml_size = 1  # PML 영역 크기
Sx = 1.0  # 높이: 위아래 pml 0부터 LN, 그 아래는 SiO2
Sy = 3.0  # 너비: 좌우 pml, 폭 3um
Sz = 3.0  # 길이 (진행방향): 3
LNwg_h = 0.5  # LN waveguide
# 설계 공간
design_region_x = LNwg_h
design_region_y = Sy
design_region_z = 2
Nx = int(resolution * design_region_x) + 1
Ny = int(resolution * design_region_y) + 1
Nz = int(resolution * design_region_z) + 1
##############################################################################################

# Numpy 배열 생성
structure_weight = np.loadtxt('lastdesign.txt')
structure_weight = structure_weight.reshape(Nx, Ny * Nz)[Nx - 1]
data = npa.rot90(structure_weight.reshape(Ny, Nz))

# 내부 채우기
filled_data = binary_fill_holes(data)  # 내부를 채움

# GDS 파일 생성
cell = gdspy.Cell('TOP')

# 직사각형의 가로와 세로 크기
width = 1
height = 1

# 경계선을 따라 외곽을 이루는 다각형(Polygon) 생성
for i in range(filled_data.shape[0]):
    for j in range(filled_data.shape[1]):
        if filled_data[i, j] == 1:
            # 직사각형의 네 꼭짓점을 이용해 외곽선을 유지
            x0, y0 = j * width, -i * height
            x1, y1 = (j + 1) * width, -(i + 1) * height
            
            # 외곽 경계선을 정의하는 좌표들
            points = [(x0, y0), (x1, y0), (x1, y1), (x0, y1)]
            polygon = gdspy.Polygon(points)
            cell.add(polygon)

# GDS 파일 저장
gdspy.write_gds('output.gds')

# Optionally, save an image of the cell as SVG.
cell.write_svg('output.svg')

# Display all cells using the internal viewer.
gdspy.LayoutViewer()


/tmp/ipykernel_2689461/2570790903.py:59: DeprecationWarning: [GDSPY] Use of the global library is deprecated.  Pass LayoutViewer a GdsLibrary instance.
  gdspy.LayoutViewer()


<gdspy.viewer.LayoutViewer object .!layoutviewer>